# NER

## Align recognized entity and vectors

In [2]:
%cd ..

/home/max/process/dist/app_noisemon


In [3]:
import spacy_alignments as tokenizations
import torch
from transformers import AutoTokenizer, AutoModel
from typing import List, Tuple

model_name: str = "cointegrated/LaBSE-en-ru"
device=torch.device("cpu")

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

model.to(device)
embedding = None
embedding_alignment = None
d = 768  # dimension

text = "alpha omega"

Some weights of the model checkpoint at cointegrated/LaBSE-en-ru were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
text

'alpha omega'

In [5]:
wordpieces

['[CLS]', 'alpha', 'omega', '[SEP]']

In [7]:
encoded_text = tokenizer([text], truncation=True, max_length=512, return_tensors="pt")
wordpieces = tokenizer.batch_decode(encoded_text.input_ids[0])
embedding_alignment, _ = tokenizations.get_alignments(list(text), wordpieces)
embedding_alignment = embedding_alignment

In [8]:
embedding_alignment

[[1], [1], [1], [1], [1], [], [2], [2], [2], [2], [2]]

In [9]:
with torch.no_grad():
    model_output = model(**{k: v.to(model.device) for k, v in encoded_text.items()})
embeddings = model_output.last_hidden_state.cpu()
embedding = torch.nn.functional.normalize(embeddings).squeeze()

In [10]:
embedding

tensor([[ 0.3372, -0.2950, -0.4121,  ..., -0.4918,  0.3570, -0.5508],
        [ 0.4493, -0.6439, -0.7166,  ..., -0.7144,  0.5637, -0.4535],
        [ 0.7555, -0.6414, -0.3833,  ..., -0.0776,  0.6538, -0.4331],
        [ 0.3372, -0.2950, -0.4121,  ..., -0.4918,  0.3570, -0.5508]])

In [11]:
char_spans = [(0,5),(6, 11)]

In [12]:
span_vectors = []
for span in char_spans:
    span_idxs = [idx for list_of_indices in embedding_alignment[span[0]: span[1]] for idx in
                 list_of_indices]
    span_idxs = sorted(set(span_idxs))
    span_emb = embedding[span_idxs]
    span_vector = torch.mean(span_emb, dim=0)
    span_vectors.append(span_vector)

In [16]:
span_emb

tensor([[ 0.7555, -0.6414, -0.3833, -0.4567,  0.5232,  0.5964,  0.4074, -0.4846,
          0.6579,  0.5078, -0.9210,  0.4519, -0.0432,  0.5566, -0.4285,  0.6470,
         -0.5387,  0.4567,  0.4417, -0.7056,  0.5970, -0.4124, -0.0942,  0.6758,
          0.3603,  0.5641,  0.1035, -0.7012,  0.3820,  0.7298, -0.5799,  0.6088,
         -0.3808,  0.1039, -0.4678, -0.1365,  0.7950, -0.6784,  0.7519,  0.6894,
         -0.0306, -0.4453,  0.7537,  0.7061, -0.5380, -0.5383,  0.5050, -0.0842,
         -0.4656,  0.6808, -0.0627,  0.4195, -0.5654,  0.7240, -0.5126,  0.2934,
         -0.0278,  0.4906,  0.5744, -0.2449, -0.6941, -0.4256,  0.3985, -0.6668,
         -0.5320,  0.5095, -0.6283,  0.6955, -0.6437, -0.1254,  0.3835, -0.7899,
          0.1866, -0.5962,  0.5720,  0.6170,  0.5044, -0.6206,  0.4666,  0.5792,
          0.0244,  0.6072, -0.7168,  0.8859, -0.9768,  0.5029,  0.2117,  0.4687,
         -0.4782, -0.4670,  0.5794, -0.5557,  0.5026,  0.4634, -0.4160, -0.8015,
          0.3124,  0.5157,  

In [15]:
embedding

tensor([[ 0.3372, -0.2950, -0.4121,  ..., -0.4918,  0.3570, -0.5508],
        [ 0.4493, -0.6439, -0.7166,  ..., -0.7144,  0.5637, -0.4535],
        [ 0.7555, -0.6414, -0.3833,  ..., -0.0776,  0.6538, -0.4331],
        [ 0.3372, -0.2950, -0.4121,  ..., -0.4918,  0.3570, -0.5508]])

In [14]:
span_idxs

[2]